In [1]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token:str, **kwargs) -> None:
        print(f"Received streaming token {token}, kwargs:{kwargs}")
        
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Received non-streaming response: {response}")

In [2]:
from llmstudio.llm.langchain import ChatLLMstudio

Running Engine on http://localhost:64493
Running Tracking on http://localhost:64494


INFO:     Started server process [36057]
INFO:     Started server process [36057]
INFO:     Waiting for application startup.
INFO:     Waiting for application startup.
INFO:     Application startup complete.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:64493 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://localhost:64494 (Press CTRL+C to quit)


In [3]:
gpt3 = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)

In [4]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def cancel_ticket(ticket_number: str):
    """Use this to cancel a ticket"""
    return "Ticket cancelled"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

@tool
def make_complaint(complaint: str):
    """Use this to forward a complaint to the complaint department"""
    return "Complaint forwarded. We appreciate your feedback."


def assistant(question: str)->str:
    """YOUR CODE HERE"""
    tools = [get_departure, cancel_ticket, buy_ticket, make_complaint]

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, gpt3, agent=AgentType.OPENAI_MULTI_FUNCTIONS, handle_parsing_errors="Check your output and make sure it conforms!", verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [5]:
assistant('When does my train depart? My ticket number is 1234')




> Entering new AgentExecutor chain...
{'id': '68f7295d-73c0-4a81-8127-e7872daa962c', 'choices': [{'finish_reason': 'function_call', 'index': 0, 'logprobs': None, 'message': {'content': None, 'role': 'assistant', 'function_call': {'arguments': '{"actions":[{"action_name":"get_departure","action":{"ticket_number":"1234"}}]}', 'name': 'tool_selection'}, 'tool_calls': None}}], 'created': 1709557781, 'model': 'gpt-3.5-turbo', 'object': 'chat.completion', 'system_fingerprint': None, 'usage': None, 'session_id': None, 'chat_input': 'When does my train depart? My ticket number is 1234', 'chat_output': '{"actions":[{"action_name":"get_departure","action":{"ticket_number":"1234"}}]}', 'context': [{'role': 'system', 'content': 'You are a helpful AI assistant.'}, {'role': 'user', 'content': 'When does my train depart? My ticket number is 1234'}], 'provider': 'openai', 'timestamp': 1709557782.38111, 'parameters': {'temperature': 0.0, 'max_tokens': None, 'top_p': None, 'frequency_penalty': 0, 'pre

{'input': 'When does my train depart? My ticket number is 1234',
 'output': 'Your train departs at 12:00 AM.'}